<a target="_blank" href="https://colab.research.google.com/github/cohere-ai/cohere-developer-experience/blob/main/notebooks/llmu/Validating_Large_Language_Model_Outputs.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Validating Large Language Model Outputs

One key property of LLMs that’s different from traditional software is that the output is probabilistic in nature. The same input (i.e., the prompt) may not always produce the same response. While this property makes it possible to build entirely new classes of natural language applications, it also means that those applications require a mechanism for validating their outputs.

An output validation step ensures that an LLM application is robust and predictable. In this article, we looked at what output validation is and how to implement it using [Guardrails AI](https://www.guardrailsai.com/).

Read the accompanying [article here](https://docs.cohere.com/docs/validating-outputs).


## 1: Setup

In [1]:
! pip install cohere git+https://github.com/guardrails-ai/guardrails.git@main

  Cloning https://github.com/guardrails-ai/guardrails.git (to revision main) to /tmp/pip-req-build-f6oew70d
  Running command git clone --filter=blob:none --quiet https://github.com/guardrails-ai/guardrails.git /tmp/pip-req-build-f6oew70d
  Resolved https://github.com/guardrails-ai/guardrails.git to commit 0134069d6e0a7c4ac74bfc0b9e7be417e6ed79a0
  Resolved https://github.com/guardrails-ai/guardrails.git to commit 0134069d6e0a7c4ac74bfc0b9e7be417e6ed79a0
  Installing build dependencies ... -  Installing build dependencies ... done
done
  Getting requirements to build wheel ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/2.0 MB ? eta -:--:--Downloading diff_match_patch-20241021-py3-none-any.whl (43 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.5 M

In [ ]:
!guardrails hub install hub://guardrails/valid_range
!guardrails hub install hub://guardrails/valid_choices

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.

Installing hub://guardrails/valid_range...

[  ==] Fetching manifest
[    ] Downloading dependencies
[    ] Running post-install setup
✅Successfully installed hub://guardrails/valid_range!

Import validator:
from guardrails.hub import ValidRange

Get more info:
https://hub.guardrailsai.com/validator/guardrails/valid_range


Installing hub://guardrails/valid_choices...

[=   ] Fetching manifest
[====] Downloading dependencies
[    ] Running post-install setup
✅Successfully installed hub://guardrails/valid_choices!

Import validator:
from guardrails.hub import ValidChoices

Get more info:
https://hub.guardrailsai.com/validator/guardrails/valid_choices



In [ ]:
import os
import cohere
import guardrails as gd
from pydantic import BaseModel, Field
from rich import print
from typing import List, Literal

# Create a Cohere client
co = cohere.Client(api_key="COHERE_API_KEY")

# Configure the API key for Guardrails
os.environ["COHERE_API_KEY"]="COHERE_API_KEY"

## 2: Define the Output Schema

Our goal is to extract detailed patient information from a medical record.
As an example, we will use the following medical record:

In [4]:
doctors_notes = """49 y/o Male with chronic macular rash to face & hair, worse in beard, eyebrows & nares.
Itchy, flaky, slightly scaly. Moderate response to OTC steroid cream"""

We want our extracted information to contain the following fields:

1. Patient's gender
2. Patient's age
3. A list of symptoms, each with a severity rating and an affected area
4. A list of medications, each with information about the patient's response to the medication

Let's define the Pydantic classes below.

In [5]:
class Symptom(BaseModel):
    symptom: str = Field(..., description="Symptom that a patient is experiencing")
    affected_area: Literal["Head", "Face", "Neck", "Chest"] = Field(
        ...,
        description="What part of the body the symptom is affecting"
    )

class CurrentMed(BaseModel):
    medication: str = Field(..., description="Name of the medication the patient is taking")
    response: str = Field(..., description="How the patient is responding to the medication")


class PatientInfo(BaseModel):
    gender: str = Field(..., description="Patient's gender")
    age: int = Field(..., description="Patient's age", ge=0, le=100)
    symptoms: List[Symptom] = Field(..., description="Symptoms that the patient is experiencing")
    current_meds: List[CurrentMed] = Field(..., description="Medications that the patient is currently taking")


## 3: Initialize a Guard Object Based on the Schema

In [ ]:
prompt_text = f"""Given the following doctor's notes about a patient,
please extract a dictionary that contains the patient's information.

{doctors_notes}
"""

In [8]:
# Initialize a Guard object from the Pydantic model PatientInfo
guard = gd.Guard.for_pydantic(output_class=PatientInfo)

## 4: Wrap an LLM Call with the Guard Object

In [ ]:
# Wrap the Cohere API call with the `guard` object
response = guard(
    messages=[
        {
            "role": "user", 
            "content": prompt_text 
        }
    ],
    model='command-a-03-2025',
    temperature=0,
    num_reasks=3,
)

# Print the validated output from the LLM
print(response.validated_output)

/home/invader/anaconda3/lib/python3.13/site-packages/guardrails/validator_service/__init__.py:84: UserWarning: Could not obtain an event loop. Falling back to synchronous validation.
  warnings.warn(


{
    'gender': 'Male',
    'age': 49,
    'symptoms': [
        {'symptom': 'chronic macular rash', 'affected_area': 'Face'},
        {'symptom': 'itchy', 'affected_area': 'Head'},
        {'symptom': 'flaky', 'affected_area': 'Head'},
        {'symptom': 'slightly scaly', 'affected_area': 'Head'}
    ],
    'current_meds': [{'medication': 'OTC steroid cream', 'response': 'moderate'}]
}

In [22]:
guard.history.last.tree

Logs
├── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
│   │ ╭─────────────────────────────────────────────── Messages ────────────────────────────────────────────────╮ │
│   │ │ ┏━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓      │ │
│   │ │ ┃ Role ┃ Content                                                                                 ┃      │ │
│   │ │ ┡━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩      │ │
│   │ │ │ user │ Given the following doctor's notes about a patient,                                     │      │ │
│   │ │ │      │ please extract a dictionary that contains the patient's information.                    │      │ │
│   │ │ │      │                                                                                         │      │ │
│   │ │ │      │ 49 y/o Male with chronic macular rash to face & hair, worse in beard, eyebrows & nares. │      │ │
│   │ │ │      │ Itchy, flaky, slightly scaly. Moderate response to OTC steroid cream                    │      │ │
│   │ │ │      │                                                                                         │      │ │
│   │ │ │      │                                                                                         │      │ │
│   │ │ │      │                                                                                         │      │ │
│   │ │ └──────┴─────────────────────────────────────────────────────────────────────────────────────────┘      │ │
│   │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│   │ ╭──────────────────────────────────────────── Raw LLM Output ─────────────────────────────────────────────╮ │
│   │ │ Here is the extracted patient information in dictionary format:                                         │ │
│   │ │                                                                                                         │ │
│   │ │ ```python                                                                                               │ │
│   │ │ {                                                                                                       │ │
│   │ │     "age": 49,                                                                                          │ │
│   │ │     "gender": "Male",                                                                                   │ │
│   │ │     "symptoms": {                                                                                       │ │
│   │ │         "condition": "chronic macular rash",                                                            │ │
│   │ │         "locations": ["face", "hair", "beard", "eyebrows", "nares"],                                    │ │
│   │ │         "characteristics": ["itchy", "flaky", "slightly scaly"]                                         │ │
│   │ │     },                                                                                                  │ │
│   │ │     "treatment": {                                                                                      │ │
│   │ │         "current": "OTC steroid cream",                                                                 │ │
│   │ │         "response": "moderate"                                                                          │ │
│   │ │     }                                                                                                   │ │
│   │ │ }                                                                                                       │ │
│   │ │ ```                                                                                                     │ │
│   │ │                                                                                                         │ │
│   │ │ This dictionary organizes the patient's age